In [2]:
from utils.metrics_utils import confusionMatrix, EncodeLabel
from utils.setup import GetAlphaFoldProteinsUsed

proteins = []
sequences = []
true_labels = []
true_labels_encoded = []
predicted_labels = []
predicted_labels_encoded = []

with open("Data/DeepTMHMM.crossval.top", "r") as fp:
    row_rel = 0
    for line in fp:
        if (line[0] == ">"):
            row_rel = 0
            # print(line[1:-1])
            proteins += [line[1:-1]]
        if row_rel == 1:
            sequences += [line[:-1]]
        if row_rel == 2:
            true_labels += [line[:-1]]
            true_labels_encoded += [EncodeLabel(line[:-1])]
        if row_rel == 3:
            predicted_labels += [line[:-1]]
            predicted_labels_encoded += [EncodeLabel(line[:-1])]
        row_rel += 1

proteinIDs = GetAlphaFoldProteinsUsed()

# Proteins in alphafold (and json) but not in crossval (are blacklisted from now on)
print(np.array(proteinIDs)[[protein not in proteins for protein in proteinIDs]])

# Filter so comparing only with the ones we are training and evaluating on (same dataset)
bool_array = [protein in proteinIDs for protein in proteins]

proteins                    = [i for indx,i in enumerate(proteins)                  if bool_array[indx] == True]
sequences                   = [i for indx,i in enumerate(sequences)                 if bool_array[indx] == True]
true_labels                 = [i for indx,i in enumerate(true_labels)               if bool_array[indx] == True]
true_labels_encoded         = [i for indx,i in enumerate(true_labels_encoded)       if bool_array[indx] == True]
predicted_labels            = [i for indx,i in enumerate(predicted_labels)          if bool_array[indx] == True]
predicted_labels_encoded    = [i for indx,i in enumerate(predicted_labels_encoded)  if bool_array[indx] == True]


c:\Users\jonas\anaconda3\envs\DeepLearningProject\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[]


In [6]:
print_error_type_pairs = True

print(f"Max protein length: {np.array([len(seq) for seq in sequences]).max()}")

(
    error_type_pairs,
    confusion_matrix,
    type_accuracy,
    detailed_type_accuracy,
) = confusionMatrix(predicted_labels_encoded, true_labels_encoded)

print(f"  type accuracy (test): {type_accuracy}")

# detailed type accuracies
for key in detailed_type_accuracy.keys():
    print(f"  {key}")
    for field in detailed_type_accuracy[key].keys():
        print(f"    {field:<9}: {detailed_type_accuracy[key][field]}")

print(confusion_matrix)

if print_error_type_pairs:
    for error_pair in error_type_pairs:
        print("  Predicted topology:", error_pair["predicted topology"])
        print("  Target topology:   ", error_pair["target topology"])

Max protein length: 1500
  type accuracy (test): 0.973730742931366
  tm
    type     : 0.9624664783477783
    topology : 0.8445039987564087
  sptm
    type     : 0.969072163105011
    topology : 0.9175257682800293
  sp
    type     : 0.9560326933860779
    topology : 0.9560326933860779
  glob
    type     : 0.9810824990272522
    topology : 0.9810824990272522
  beta
    type     : 1.0
    topology : 0.8101266026496887
tensor([[  44,    7,    2,    5,    0,  315,    0],
        [   0,    5,    2,    1,    0,   89,    0],
        [   0,   23,    0,   11,    7,  935,    2],
        [  17,    0,    8,    0,    0, 1867,   11],
        [   0,    0,    0,    0,   15,   64,    0]])
  Predicted topology: OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
  Target topology:    SSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO